In [1]:
import pandas as pd 
import numpy_financial as npf
from datetime import date, datetime
import plotly.express as px

In [2]:
loans = {}

In [66]:
# variables Turnstone 
loans['Turnstone'] = {
"interest" :0.03125,
"years":30, 
"payments_year":12,
"mortgage":436000,
"start_date":(date(2020, 12, 1)),
"term":"full"
}
# variables Heloc  
loans['Heloc'] = {
"interest" :0.066369,
# "interest" :0.0625,
"years":20, 
"payments_year":12,
# "mortgage":98039.88,
"mortgage":100921.88,    
"start_date":(date(2022, 9, 1)),
"term":"int_only"
}
# variables Lakeside  
loans['LakeSide'] = {
"interest" :0.0575,
"years":15, 
"payments_year":12,
"mortgage":86550.00,
"start_date":(date(2022, 11, 1)),
"payback_years":5,
"term":"jumbo"
}
# variables Brier Rose  
loans['BrierRose'] = {
"interest" :0.03375,
"years":30, 
"payments_year":12,
"mortgage":277520,
"start_date":(date(2021, 3, 1)),
"Current_payment":1767.56,
"HOA_club":161.25,
"HOA_resort":204.00,
"term":"full"
}

In [299]:
for m in loans:
    print(m)

Turnstone
Heloc
LakeSide
BrierRose


In [300]:
schedules = {}
for m in loans:
    if loans[m]['term'] == 'full_amtor':
        schedules[m]=fully_amtr_sched(loans[m], extra_payment= None)
    if loans[m]['term'] == 'interest_only':
        schedules[m]=io_sched(loans[m])        
    if loans[m]['term'] == 'jumbo':
        print("jumbo has not been created yet")

jumbo has not been created yet


# Full Amortization schedule

In [64]:
def fully_amtr_sched(property_vars, extra_payment= None):
    interest = property_vars['interest']
    years = property_vars['years']
    payments_year = property_vars['payments_year']
    mortgage = property_vars['mortgage']
    start_date =property_vars['start_date']  
    mortgage_type =property_vars['term']
    initial_additional_pmt = 0
    
    
    # initial values for monthly payment, interest and principal
    initial_pmt = -1 * npf.pmt(interest/12, years*payments_year, mortgage)
    initial_ipmt = -1 * npf.ipmt(interest/payments_year,1, years*payments_year, mortgage)
    initial_ppmt = -1 * npf.ppmt(interest/payments_year, 1,years*payments_year, mortgage)
    
    # create DF
    if mortgage_type == 'full':
        rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
        rng.name = "Payment Date"
        # create dataframe 
        full_df = pd.DataFrame(
            index= rng,
            columns=['Org Total Payment','Total Payment','Interest','Principal',
                     'Additional Payment','Org Ending Balance','Ending Balance'], dtype='float')
        full_df.reset_index(inplace=True)
        full_df.index += 1
        full_df.index.name ="Period"
        
    if mortgage_type == 'jumbo':        
        rng = pd.date_range(start_date, periods=property_vars["payback_years"] * payments_year, freq='MS')
        rng.name = "Payment Date"
        # create dataframe 
        full_df = pd.DataFrame(
            index= rng,
            columns=['Org Total Payment','Total Payment','Interest','Principal',
                     'Additional Payment','Org Ending Balance','Ending Balance'], dtype='float')
        full_df.reset_index(inplace=True)
        full_df.index += 1
        full_df.index.name ="Period"
        
    if mortgage_type == 'int_only':        
        rng = pd.date_range(start_date, periods=property_vars["payback_years"] * payments_year, freq='MS')
        rng.name = "Payment Date"
        # create dataframe 
        full_df = pd.DataFrame(
            index= rng,
            columns=['Org Total Payment','Total Payment','Interest','Principal',
                     'Additional Payment','Org Ending Balance','Ending Balance'], dtype='float')
        full_df.reset_index(inplace=True)
        full_df.index += 1
        full_df.index.name ="Period"

    
    if extra_payment != None:
        period_additional_pmt = initial_additional_pmt
    else:
        period_additional_pmt = 0     
       
    # Create values for the first period
    period = 1
    # for each element in the row set the value
    initial_row_dict = {
        'Org Total Payment': initial_pmt,
        'Total Payment': initial_pmt + (initial_additional_pmt),
        'Interest': initial_ipmt,
        'Principal': initial_ppmt,
#         'Rent': start_rent,
#         'Cash Flow': start_cash_flow,
        'Additional Payment': initial_additional_pmt, 
        'Org Ending Balance': mortgage - initial_ppmt,
        'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)    
    }
    columns = list(initial_row_dict.keys())
    period_values = list(initial_row_dict.values())
    full_df.at[period, columns] = period_values
    # round values 
    full_df = full_df.round(2)
    
    # Add the rest of the rows 
    for period in range(2, len(full_df)+1):
        # get the prior period values 

        previous_total_payments = full_df.loc[period-1,'Total Payment']
        previous_principal = full_df.loc[period-1,'Principal']
        previous_org_ending_balance = full_df.loc[period-1,'Org Ending Balance']
        previous_ending_balance = full_df.loc[period-1,'Ending Balance']
        previous_= full_df.loc[period-1,'Total Payment']
        
        ######
        # Check if Jumbo and add lump due if period met
        ######
        if property_vars['term'] == 'jumbo':
#             print('This is a jumbo loan')
            final_period = property_vars['payback_years'] * 12
#             print("final payment due on period "+str(final_period)  )
            if period == final_period:
                print("Should make final payment here")
                extra_payment = previous_ending_balance
                initial_additional_pmt = previous_ending_balance
                print("Payment amount "+ str(extra_payment))
        
#         try:
#             print('Extra Payment Set to '+str(extra_payment))
        # set additional payment values 
        if extra_payment != None:
#             print("Setting additional payment")
            period_additional_pmt = initial_additional_pmt
            
        else:
            period_additional_pmt = 0   

       # Calculate remainder values to get the end of loan bal
        period_interest = previous_org_ending_balance * interest / payments_year
        period_principal = initial_pmt - period_interest
        org_ending_balance = previous_org_ending_balance - period_principal
        ending_balance = previous_ending_balance - period_principal - period_additional_pmt
        org_ending_balance = 0 if org_ending_balance <= 0 else org_ending_balance
        ending_balance = 0 if ending_balance <= 0 else ending_balance

        row_dict ={
        'Org Total Payment': initial_pmt,
        'Total Payment': initial_pmt + period_additional_pmt,
        'Interest': period_interest,
        'Principal': period_principal,
        'Additional Payment': period_additional_pmt, 
        'Org Ending Balance': org_ending_balance,
        'Ending Balance': ending_balance
        }
        columns = list(row_dict.keys())
        period_values = list(row_dict.values())
        full_df.loc[period, columns] = period_values
        full_df = full_df.round(2)
    if property_vars['term'] == 'jumbo':
        full_df = full_df[:final_period]
    return full_df

# Jumbo

In [21]:
loans['LakeSide']

{'interest': 0.03375,
 'years': 10,
 'payments_year': 12,
 'mortgage': 86550.0,
 'start_date': datetime.date(2022, 11, 1),
 'payback_years': 5,
 'term': 'jumbo'}

In [27]:
# variables Lakeside  
loans['LakeSide'] = {
"interest" :0.0575,
"years":15, 
"payments_year":12,
"mortgage":86550.00,
"start_date":(date(2022, 11, 1)),
"payback_years":5,
"term":"jumbo"
}

In [65]:
fully_amtr_sched(loans['LakeSide'], extra_payment= None)

Should make final payment here
Payment amount 65878.59


,Payment Date,Org Total Payment,Total Payment,Interest,Principal,Additional Payment,Org Ending Balance,Ending Balance
Period,,,,,,,,
1,2022-11-01,718.72,718.72,414.72,304.00,0.00,86246.00,86246.00
2,2022-12-01,718.72,718.72,413.26,305.46,0.00,85940.54,85940.54
3,2023-01-01,718.72,718.72,411.80,306.92,0.00,85633.62,85633.62
4,2023-02-01,718.72,718.72,410.33,308.39,0.00,85325.23,85325.23
5,2023-03-01,718.72,718.72,408.85,309.87,0.00,85015.36,85015.36
6,2023-04-01,718.72,718.72,407.37,311.35,0.00,84704.01,84704.01
7,2023-05-01,718.72,718.72,405.87,312.85,0.00,84391.16,84391.16
8,2023-06-01,718.72,718.72,404.37,314.35,0.00,84076.81,84076.81
9,2023-07-01,718.72,718.72,402.87,315.85,0.00,83760.96,83760.96


In [39]:
final_period = loans['LakeSide']['payback_years'] * 12

In [41]:
final_due=full.loc[final_period-1]['Ending Balance']

In [60]:
fully_amtr_sched(loans['LakeSide'], extra_payment= None,final_period)

SyntaxError: positional argument follows keyword argument (1341342667.py, line 1)

# Interest Only

In [ ]:
def io_sched(pl_json,extra_payment= None):
    interest = pl_json['interest']
    years = pl_json['years']
    payments_year = pl_json['payments_year']
    mortgage = pl_json['mortgage']
    start_date =pl_json['start_date'] 
    
    initial_ipmt = -1 * npf.ipmt(interest/payments_year,1, years*payments_year, mortgage)
    initial_additional_pmt = 0
    
    # create DF 
    rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
    rng.name = "Payment Date"
    # create dataframe 
    io_df = pd.DataFrame(
        index= rng,
        columns=['Org Total Payment','Total Payment','Interest','Principal',
                 'Additional Payment','Org Ending Balance','Ending Balance'], dtype='float')
    io_df.reset_index(inplace=True)
    io_df.index += 1
    io_df.index.name ="Period"
    # Create values for the first period
    period = 1
    # for each element in the row set the value
    initial_row_dict = {
        'Org Total Payment': initial_ipmt,
        'Total Payment': initial_ipmt + (initial_additional_pmt),
        'Interest': initial_ipmt,
        'Principal': initial_additional_pmt,
        'Additional Payment': initial_additional_pmt, 
        'Org Ending Balance': mortgage - initial_ppmt,
        'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)    
    }
    columns = list(initial_row_dict.keys())
    period_values = list(initial_row_dict.values())
    io_df.at[period, columns] = period_values
    # round values 
    io_df = io_df.round(2)
    # Add the rest of the rows 
    for period in range(2, len(io_df)+1):
        # get the prior period values 
        previous_total_payments = io_df.loc[period-1,'Total Payment']
        previous_principal = io_df.loc[period-1,'Principal']
        previous_org_ending_balance = io_df.loc[period-1,'Org Ending Balance']
        previous_ending_balance = io_df.loc[period-1,'Ending Balance']
        previous_= io_df.loc[period-1,'Total Payment']

        if extra_payment != None:
            period_additional_pmt = initial_additional_pmt
        else:
            period_additional_pmt = 0 

        # Calculate remainder values to get the end of loan bal
        period_interest = previous_org_ending_balance * interest / payments_year
        period_principal = initial_pmt - period_interest
        org_ending_balance = previous_org_ending_balance - period_principal
        ending_balance = previous_ending_balance - period_principal - period_additional_pmt
        org_ending_balance = 0 if org_ending_balance <= 0 else org_ending_balance
        ending_balance = 0 if ending_balance <= 0 else ending_balance

        row_dict ={
        'Org Total Payment': initial_ipmt,
        'Total Payment': initial_ipmt + (initial_additional_pmt),
        'Interest': initial_ipmt,
        'Principal': initial_additional_pmt,
        'Additional Payment': initial_additional_pmt, 
        'Org Ending Balance': mortgage - initial_ppmt,
        'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)  
        }
        columns = list(row_dict.keys())
        period_values = list(row_dict.values())
        io_df.loc[period, columns] = period_values
        io_df = io_df.round(2)
    
    return io_df

# Testing

In [18]:
beacon_sav=2880.50

In [19]:
BR_payment=BrierRose['Current_payment']+BrierRose['HOA_club']+BrierRose['HOA_resort']
BR_annual_rev=40000.00
monthMaint=600.00
mgmt_fees=BR_annual_rev*.15

In [12]:
profit=BR_annual_rev-(monthMaint*12)-mgmt_fees-(BR_payment*12)
profit

1206.2799999999988

6000.0

In [51]:
mike_month=1185.25 * 2
 

In [52]:
# Scenario #1
extra_payment = mike_month

In [122]:
# Scenario #2 
# Investor/LandLord: Dynamic payment base on % of rental cash flow
start_rent = None
start_cash_flow = None 
rent_increase_yoy = None 
extra_payment_prct = None

In [22]:
turn_df=get_mortgage_amoritization(Turnstone)
turn_df

,Payment Date,Org Total Payment,Total Payment,Interest,Principal,Rent,Cash Flow,Additional Payment,Org Ending Balance,Ending Balance
Period,,,,,,,,,,
1,2020-12-01,1867.72,1867.72,1135.42,732.30,NaN,NaN,0.0,435267.70,435267.70
2,2021-01-01,1867.72,1867.72,1133.51,734.21,0.0,0.0,0.0,434533.49,434533.49
3,2021-02-01,1867.72,1867.72,1131.60,736.12,0.0,0.0,0.0,433797.37,433797.37
4,2021-03-01,1867.72,1867.72,1129.68,738.04,0.0,0.0,0.0,433059.33,433059.33
5,2021-04-01,1867.72,1867.72,1127.76,739.96,0.0,0.0,0.0,432319.37,432319.37
...,...,...,...,...,...,...,...,...,...,...
356,2050-07-01,1867.72,1867.72,24.13,1843.59,0.0,0.0,0.0,7422.41,7422.41
357,2050-08-01,1867.72,1867.72,19.33,1848.39,0.0,0.0,0.0,5574.02,5574.02
358,2050-09-01,1867.72,1867.72,14.52,1853.20,0.0,0.0,0.0,3720.82,3720.82


In [55]:
br_df=get_mortgage_amoritization(BrierRose,mike_month)

In [127]:
df

,Payment Date,Org Total Payment,Total Payment,Interest,Principal,Rent,Cash Flow,Additional Payment,Org Ending Balance,Ending Balance
Period,,,,,,,,,,
1,2020-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
356,2050-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,2050-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,2050-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Visualization

In [25]:
df_plot = pd.melt(turn_df, id_vars=['Payment Date'], value_vars=['Org Ending Balance', 'Ending Balance'])
df_plot

,Payment Date,variable,value
0,2020-12-01,Org Ending Balance,435267.70
1,2021-01-01,Org Ending Balance,434533.49
2,2021-02-01,Org Ending Balance,433797.37
3,2021-03-01,Org Ending Balance,433059.33
4,2021-04-01,Org Ending Balance,432319.37
...,...,...,...
715,2050-07-01,Ending Balance,7422.41
716,2050-08-01,Ending Balance,5574.02
717,2050-09-01,Ending Balance,3720.82
718,2050-10-01,Ending Balance,1862.79


In [7]:
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.0f%s' % (round(num), ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

In [60]:
# create plotly chart
fig = px.line(df_plot, x='Payment Date', y='value', color='variable')
fig.add_vline(x=early_payment_date, line_dash="dot", line_color="black")
fig.add_vrect(
  x0=early_payment_date, 
  x1=max_payment_date, 
  fillcolor="red", 
  opacity=0.25, 
  annotation_position="top left",
  annotation_text="+${0} interest savings".format(human_format(additional_interest))
)
fig.update_layout(
  title='Mortgage Amoritization', 
  xaxis_title='Year', 
  yaxis_title='Mortgage Balance', 
  plot_bgcolor='white',
  legend=dict(
    title_text="Balance Type",
    yanchor="bottom",
    y=0.02,
    xanchor="left",
    x=0.01)
)
fig.show()

# New Visualization

In [45]:
# br_df

In [47]:
frames = [turn_df,br_df]
df = pd.concat(frames)

In [50]:
df_plot = pd.melt(df, id_vars=['Payment Date'], value_vars=['Org Ending Balance', 'Ending Balance'])
# br_plot = pd.melt(br_df, id_vars=['Payment Date'], value_vars=['Org Ending Balance', 'Ending Balance'])
# create plotly chart
fig = px.line(df_plot, x='Payment Date', y='value', color='variable')
# fig = px.line(br_plot, x='Payment Date', y='value', color='variable')

fig.update_layout(
  title='Mortgage Amoritization', 
  xaxis_title='Year', 
  yaxis_title='Mortgage Balance', 
  plot_bgcolor='white',
  legend=dict(
    title_text="Balance Type",
    yanchor="bottom",
    y=0.02,
    xanchor="left",
    x=0.01)
)
fig.show()

# Functions 

In [11]:
def io_sched(pl_json):
    interest = pl_json['interest']
    years = pl_json['years']
    payments_year = pl_json['payments_year']
    mortgage = pl_json['mortgage']
    start_date =pl_json['start_date'] 
    
    initial_ipmt = -1 * npf.ipmt(interest/payments_year,1, years*payments_year, mortgage)
    initial_additional_pmt = 0
    extra_payment= None
    # create DF 
    rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
    rng.name = "Payment Date"
    # create dataframe 
    io_df = pd.DataFrame(
        index= rng,
        columns=['Org Total Payment','Total Payment','Interest','Principal',
                 'Additional Payment','Org Ending Balance','Ending Balance'], dtype='float')
    io_df.reset_index(inplace=True)
    io_df.index += 1
    io_df.index.name ="Period"
    # Create values for the first period
    period = 1
    # for each element in the row set the value
    initial_row_dict = {
        'Org Total Payment': initial_ipmt,
        'Total Payment': initial_ipmt + (initial_additional_pmt),
        'Interest': initial_ipmt,
        'Principal': initial_additional_pmt,
        'Additional Payment': initial_additional_pmt, 
        'Org Ending Balance': mortgage - initial_ppmt,
        'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)    
    }
    columns = list(initial_row_dict.keys())
    period_values = list(initial_row_dict.values())
    io_df.at[period, columns] = period_values
    # round values 
    io_df = io_df.round(2)
    # Add the rest of the rows 
    for period in range(2, len(io_df)+1):
        # get the prior period values 
        previous_total_payments = io_df.loc[period-1,'Total Payment']
        previous_principal = io_df.loc[period-1,'Principal']
        previous_org_ending_balance = io_df.loc[period-1,'Org Ending Balance']
        previous_ending_balance = io_df.loc[period-1,'Ending Balance']
        previous_= io_df.loc[period-1,'Total Payment']

        if extra_payment != None:
            period_additional_pmt = initial_additional_pmt
        else:
            period_additional_pmt = 0 


        # Calculate remainder values to get the end of loan bal
        period_interest = previous_org_ending_balance * interest / payments_year
        period_principal = initial_pmt - period_interest
        org_ending_balance = previous_org_ending_balance - period_principal
        ending_balance = previous_ending_balance - period_principal - period_additional_pmt
        org_ending_balance = 0 if org_ending_balance <= 0 else org_ending_balance
        ending_balance = 0 if ending_balance <= 0 else ending_balance

        row_dict ={
        'Org Total Payment': initial_ipmt,
        'Total Payment': initial_ipmt + (initial_additional_pmt),
        'Interest': initial_ipmt,
        'Principal': initial_additional_pmt,
        'Additional Payment': initial_additional_pmt, 
        'Org Ending Balance': mortgage - initial_ppmt,
        'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)  
        }
        columns = list(row_dict.keys())
        period_values = list(row_dict.values())
        io_df.loc[period, columns] = period_values
        io_df = io_df.round(2)
    
    return io_df

def fully_amtr_sched(property_vars, extra_payment= None):
    interest = property_vars['interest']
    years = property_vars['years']
    payments_year = property_vars['payments_year']
    mortgage = property_vars['mortgage']
    start_date =property_vars['start_date']    

    
    # initial values for monthly payment, interest and principal
    initial_pmt = -1 * npf.pmt(interest/12, years*payments_year, mortgage)
    initial_ipmt = -1 * npf.ipmt(interest/payments_year,1, years*payments_year, mortgage)
    initial_ppmt = -1 * npf.ppmt(interest/payments_year, 1,years*payments_year, mortgage)
    initial_additional_pmt = 0
    # create DF 
    rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
    rng.name = "Payment Date"
    # create dataframe 
    full_df = pd.DataFrame(
        index= rng,
        columns=['Org Total Payment','Total Payment','Interest','Principal',
                 'Additional Payment','Org Ending Balance','Ending Balance'], dtype='float')
    full_df.reset_index(inplace=True)
    full_df.index += 1
    full_df.index.name ="Period"

    if extra_payment != None:
        period_additional_pmt = initial_additional_pmt
    else:
        period_additional_pmt = 0     
       
    # Create values for the first period
    period = 1
    # for each element in the row set the value
    initial_row_dict = {
        'Org Total Payment': initial_pmt,
        'Total Payment': initial_pmt + (initial_additional_pmt),
        'Interest': initial_ipmt,
        'Principal': initial_ppmt,
#         'Rent': start_rent,
#         'Cash Flow': start_cash_flow,
        'Additional Payment': initial_additional_pmt, 
        'Org Ending Balance': mortgage - initial_ppmt,
        'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)    
    }
    columns = list(initial_row_dict.keys())
    period_values = list(initial_row_dict.values())
    full_df.at[period, columns] = period_values
    # round values 
    full_df = full_df.round(2)
    
    # Add the rest of the rows 
    for period in range(2, len(full_df)+1):
        # get the prior period values 

        previous_total_payments = full_df.loc[period-1,'Total Payment']
        previous_principal = full_df.loc[period-1,'Principal']
        previous_org_ending_balance = full_df.loc[period-1,'Org Ending Balance']
        previous_ending_balance = full_df.loc[period-1,'Ending Balance']
        previous_= full_df.loc[period-1,'Total Payment']

        # set additional payment values 
        if extra_payment != None:
            period_additional_pmt = initial_additional_pmt
        else:
            period_additional_pmt = 0   

       # Calculate remainder values to get the end of loan bal
        period_interest = previous_org_ending_balance * interest / payments_year
        period_principal = initial_pmt - period_interest
        org_ending_balance = previous_org_ending_balance - period_principal
        ending_balance = previous_ending_balance - period_principal - period_additional_pmt
        org_ending_balance = 0 if org_ending_balance <= 0 else org_ending_balance
        ending_balance = 0 if ending_balance <= 0 else ending_balance

        row_dict ={
        'Org Total Payment': initial_pmt,
        'Total Payment': initial_pmt + period_additional_pmt,
        'Interest': period_interest,
        'Principal': period_principal,
        'Additional Payment': period_additional_pmt, 
        'Org Ending Balance': org_ending_balance,
        'Ending Balance': ending_balance
        }
        columns = list(row_dict.keys())
        period_values = list(row_dict.values())
        full_df.loc[period, columns] = period_values
        full_df = full_df.round(2)
    return full_df
    

In [205]:
# initial values for monthly payment, interest and principal
initial_pmt = -1 * npf.pmt(interest/12, years*payments_year, mortgage)
initial_ipmt = -1 * npf.ipmt(interest/payments_year,1, years*payments_year, mortgage)
initial_ppmt = -1 * npf.ppmt(interest/payments_year, 1,years*payments_year, mortgage)
initial_additional_pmt = 0
extra_payment= None

In [207]:
print(initial_pmt)
print(initial_ipmt)
print(initial_ppmt)

760.602452344057
558.17368781
202.428764534057


In [208]:
# create DF 
rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
rng.name = "Payment Date"
# create dataframe 
io_df = pd.DataFrame(
    index= rng,
    columns=['Org Total Payment','Total Payment','Interest','Principal',
             'Additional Payment','Org Ending Balance','Ending Balance'], dtype='float')
io_df.reset_index(inplace=True)
io_df.index += 1
io_df.index.name ="Period"

In [215]:
# Create values for the first period
period = 1
# for each element in the row set the value
initial_row_dict = {
    'Org Total Payment': initial_ipmt,
    'Total Payment': initial_ipmt + (initial_additional_pmt),
    'Interest': initial_ipmt,
    'Principal': initial_additional_pmt,
    'Additional Payment': initial_additional_pmt, 
    'Org Ending Balance': mortgage - initial_ppmt,
    'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)    
}
columns = list(initial_row_dict.keys())
period_values = list(initial_row_dict.values())
io_df.at[period, columns] = period_values
# round values 
io_df = io_df.round(2)


In [218]:
# Add the rest of the rows 
for period in range(2, len(io_df)+1):
    # get the prior period values 
    previous_total_payments = df.loc[period-1,'Total Payment']
    previous_principal = df.loc[period-1,'Principal']
    previous_rent = df.loc[period-1,'Rent']
    previous_cf = df.loc[period-1,'Cash Flow']
    previous_org_ending_balance = df.loc[period-1,'Org Ending Balance']
    previous_ending_balance = df.loc[period-1,'Ending Balance']
    previous_= df.loc[period-1,'Total Payment']
       
    if extra_payment != None:
        period_additional_pmt = initial_additional_pmt
    else:
        period_additional_pmt = 0 
#         period_rent = 0 
#         period_cash_flow = 0
#         extra_payment_prct = 0
    
    # Calculate remainder values to get the end of loan bal
    period_interest = previous_org_ending_balance * interest / payments_year
    period_principal = initial_pmt - period_interest
    org_ending_balance = previous_org_ending_balance - period_principal
    ending_balance = previous_ending_balance - period_principal - period_additional_pmt
    org_ending_balance = 0 if org_ending_balance <= 0 else org_ending_balance
    ending_balance = 0 if ending_balance <= 0 else ending_balance
    
    row_dict ={
    'Org Total Payment': initial_ipmt,
    'Total Payment': initial_ipmt + (initial_additional_pmt),
    'Interest': initial_ipmt,
    'Principal': initial_additional_pmt,
    'Additional Payment': initial_additional_pmt, 
    'Org Ending Balance': mortgage - initial_ppmt,
    'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)  
    }
    columns = list(row_dict.keys())
    period_values = list(row_dict.values())
    io_df.loc[period, columns] = period_values
    io_df = io_df.round(2)



In [222]:
io_df

,Payment Date,Org Total Payment,Total Payment,Interest,Principal,Additional Payment,Org Ending Balance,Ending Balance
Period,,,,,,,,
1,2022-09-01,558.17,558.17,558.17,0.0,0.0,100719.45,100719.45
2,2022-10-01,558.17,558.17,558.17,0.0,0.0,100719.45,100719.45
3,2022-11-01,558.17,558.17,558.17,0.0,0.0,100719.45,100719.45
4,2022-12-01,558.17,558.17,558.17,0.0,0.0,100719.45,100719.45
5,2023-01-01,558.17,558.17,558.17,0.0,0.0,100719.45,100719.45
...,...,...,...,...,...,...,...,...
236,2042-04-01,558.17,558.17,558.17,0.0,0.0,100719.45,100719.45
237,2042-05-01,558.17,558.17,558.17,0.0,0.0,100719.45,100719.45
238,2042-06-01,558.17,558.17,558.17,0.0,0.0,100719.45,100719.45


In [227]:
    row_dict ={
    'Org Total Payment': initial_pmt,
    'Total Payment': initial_pmt + period_additional_pmt,
    'Interest': period_interest,
    'Principal': period_principal,
    'Rent': period_rent,
    'Cash Flow': period_cash_flow,
    'Additional Payment': period_additional_pmt, 
    'Org Ending Balance': org_ending_balance,
    'Ending Balance': ending_balance
    }

In [228]:
row_dict

{'Org Total Payment': 760.602452344057,
 'Total Payment': 760.602452344057,
 'Interest': nan,
 'Principal': nan,
 'Rent': 0,
 'Cash Flow': 0,
 'Additional Payment': 0,
 'Org Ending Balance': nan,
 'Ending Balance': nan}

0